In [1]:
from gdc.data_access import *
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import r2_score, mean_squared_error
from arch import arch_model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
df_load_simulated_normalized.head()[[0, 1, 2]]

,0,1,2
2023-01-01 00:00:00,0.308875,0.241084,0.435789
2023-01-01 01:00:00,0.325552,0.246717,0.424862
2023-01-01 02:00:00,3.468337,0.245007,2.300304
2023-01-01 03:00:00,2.215840,1.272533,2.411101
2023-01-01 04:00:00,1.259481,1.027133,0.483457


In [3]:
df_temp_simulated_normalized.head()[[0, 1, 2]]

,0,1,2
2023-01-01 00:00:00,13.021741,7.970941,7.970941
2023-01-01 01:00:00,12.621741,7.970941,7.970941
2023-01-01 02:00:00,12.621741,8.070941,8.070941
2023-01-01 03:00:00,12.421742,8.070941,8.070941
2023-01-01 04:00:00,12.321742,7.870941,7.870941


In [4]:
def print_coeffs_and_forecast_metrics(
    Yv, HDDv, CDDv, m_idx, beta,
    alpha_m=None, alpha_im=None,
    alpha_ih=None,            # (24,N)
    Hy=None,                  # pooled hour (24,N) — unused for this model
    Dy=None,                  # pooled DOW  (7,N)  — unused for this model
    alpha_id=None,            # <-- NEW: (7,N) i×DOW
    rho=None, hod=None, dow=None,
    label="Model"
):
    mu = np.zeros_like(Yv, dtype=float)
    if alpha_m is not None:
        mu += alpha_m[m_idx, None]
    if alpha_im is not None:
        mu += alpha_im[m_idx, :]
    if hod is not None:
        if alpha_ih is not None:
            mu += alpha_ih[hod, :]
        elif Hy is not None:
            mu += Hy[hod, :]
    if dow is not None:
        if alpha_id is not None:
            mu += alpha_id[dow, :]
        elif Dy is not None:
            mu += Dy[dow, :]
    mu += beta[0]*HDDv + beta[1]*CDDv

    resid_s = Yv - mu
    sse_s = float(np.sum(resid_s**2))
    sst   = float(np.sum((Yv - Yv.mean())**2))
    r2_s  = 1.0 - sse_s / sst
    rmse_s = float(np.sqrt(sse_s / Yv.size))

    # --- dynamic one-step fit (if rho provided) ---
    if rho is not None:
        yhat_dyn = mu.copy()
        yhat_dyn[1:, :] += rho * (Yv[:-1, :] - mu[:-1, :])
        resid_d = Yv - yhat_dyn
        sse_d = float(np.sum(resid_d**2))
        r2_d  = 1.0 - sse_d / sst
        rmse_d = float(np.sqrt(sse_d / Yv.size))
    else:
        r2_d = rmse_d = None

    # --- print summary ---
    print(f"\n{label}")
    print("-" * len(label))
    print(f"β_HDD = {beta[0]:.4f}")
    print(f"β_CDD = {beta[1]:.4f}")
    print("Static (mean) fit:")
    print(f"  R²   = {r2_s:.4f}")
    print(f"  RMSE = {rmse_s:.4f}")
    if rho is not None:
        print("\nDynamic one-step forecast (AR1):")
        print(f"  ρ    = {rho:.4f}")
        print(f"  R²   = {r2_d:.4f}")
        print(f"  RMSE = {rmse_d:.4f}")


In [5]:
Y = df_load_simulated_normalized
T = df_temp_simulated_normalized

In [6]:
tau_h, tau_c = 15.0, 20.0
HDD = (tau_h - T).clip(lower=0)
CDD = (T - tau_c).clip(lower=0)

month = Y.index.month.values              # (nT,)
m_idx = month - 1                         # 0..11
nT, nI = Y.shape
months = np.arange(12)

# Cast once if you want memory speedups
Yv   = Y.to_numpy(dtype=np.float32, copy=False)
HDDv = HDD.to_numpy(dtype=np.float32, copy=False)
CDDv = CDD.to_numpy(dtype=np.float32, copy=False)

In [7]:
# indices (assuming regular hourly grid)
Tn, N = Yv.shape
idx = np.arange(Tn)
hod = idx % 24                    # 0..23
dow = (idx // 24) % 7             # 0..6
months = np.arange(12)            # 0..11


In [8]:
def pooled_tables_centered(Yv, hod, dow):
    """
    Returns centered pooled seasonal intercept tables:
      Hy0: (24, N) hour-of-day FE, column-centered (sum over 24 = 0 per consumer)
      Dy0: ( 7, N) day-of-week FE, column-centered (sum over 7  = 0 per consumer)
    """
    Hy  = np.vstack([Yv[hod==h, :].mean(axis=0) for h in range(24)])  # (24,N)
    Dy  = np.vstack([Yv[dow==d, :].mean(axis=0) for d in range(7)])   # (7,N)
    Hy0 = Hy - Hy.mean(axis=0, keepdims=True)
    Dy0 = Dy - Dy.mean(axis=0, keepdims=True)
    return Hy0, Dy0



### Model A — H + DOW + Month FE (pooled) + iid

cons ~ temp_low + temp_high 

In [9]:
def fit_model_A_with_pooled_seasonal(Yv, HDDv, CDDv, m_idx, hod, dow):
    # centered pooled seasonal intercepts
    Hy0, Dy0 = pooled_tables_centered(Yv, hod, dow)

    # pooled month means (scalars per month)
    mY = np.array([Yv[m_idx==k, :].mean()  for k in range(12)])
    mH = np.array([HDDv[m_idx==k,:].mean() for k in range(12)])
    mC = np.array([CDDv[m_idx==k,:].mean() for k in range(12)])

    # within by month on Y and X; subtract pooled seasonal from Y only
    Yw   = (Yv   - mY[m_idx, None]) - Hy0[hod, :] - Dy0[dow, :]
    HDDw = (HDDv - mH[m_idx, None])
    CDDw = (CDDv - mC[m_idx, None])

    # 2×2 normal equations
    Shh = np.einsum('ij,ij->', HDDw, HDDw); Scc = np.einsum('ij,ij->', CDDw, CDDw)
    Shc = np.einsum('ij,ij->', HDDw, CDDw)
    Shy = np.einsum('ij,ij->', HDDw, Yw);   Scy = np.einsum('ij,ij->', CDDw, Yw)
    det = Shh*Scc - Shc*Shc
    beta_A = np.array([(Shy*Scc - Scy*Shc)/det, (-Shy*Shc + Scy*Shh)/det], dtype=np.float64)

    # month intercepts (on original scale)
    alpha_m = mY - (beta_A[0]*mH + beta_A[1]*mC)

    seasonal = {"Hy0": Hy0, "Dy0": Dy0}
    return alpha_m, beta_A, seasonal

alpha_m, beta_A, seasonal = fit_model_A_with_pooled_seasonal(Yv, HDDv, CDDv, m_idx, hod, dow)

In [10]:
Hy = seasonal['Hy0']
Dy = seasonal['Dy0']

print_coeffs_and_forecast_metrics(
    Yv, HDDv, CDDv, m_idx, beta_A, alpha_m=alpha_m, alpha_im=None,
    alpha_ih=None, Hy=Hy, Dy=Dy, rho=None, hod=hod, dow=dow,
    label="Model A"
)


Model A
-------
β_HDD = 0.0231
β_CDD = 0.0078
Static (mean) fit:
  R²   = 0.2778
  RMSE = 0.7656


### Model B — Month x Individual FE + uncorrelated errors

cons ~ temp_low + temp_high

In [11]:
# def fit_model_B(Yv, HDDv, CDDv, m_idx):
#     months = np.arange(12)
#     # monthly means per consumer (12×N)
#     My = np.vstack([Yv[m_idx==k,:].mean(axis=0)   for k in months]).astype(np.float64)
#     Mh = np.vstack([HDDv[m_idx==k,:].mean(axis=0) for k in months]).astype(np.float64)
#     Mc = np.vstack([CDDv[m_idx==k,:].mean(axis=0) for k in months]).astype(np.float64)

#     # within (i,m) transform
#     Yw   = Yv   - My[m_idx, :]
#     HDDw = HDDv - Mh[m_idx, :]
#     CDDw = CDDv - Mc[m_idx, :]

#     # 2×2 normal equations
#     Shh = np.einsum('ij,ij->', HDDw, HDDw, optimize=True)
#     Scc = np.einsum('ij,ij->', CDDw, CDDw, optimize=True)
#     Shc = np.einsum('ij,ij->', HDDw, CDDw, optimize=True)
#     Shy = np.einsum('ij,ij->', HDDw, Yw,   optimize=True)
#     Scy = np.einsum('ij,ij->', CDDw, Yw,   optimize=True)
#     det = Shh*Scc - Shc*Shc
#     beta_B = np.array([( Shy*Scc - Scy*Shc)/det,
#                        (-Shy*Shc + Scy*Shh)/det], dtype=np.float64)

#     # intercepts α_{i,m}
#     alpha_im_B = (My - (beta_B[0]*Mh + beta_B[1]*Mc)).astype(np.float64)
#     return alpha_im_B, beta_B, (My, Mh, Mc)  # return means for later reuse

# alpha_im_B, beta_B, MyMhMc = fit_model_B(Yv, HDDv, CDDv, m_idx)

def fit_model_B_with_pooled_seasonal(Yv, HDDv, CDDv, m_idx, hod, dow):
    # per-(i,month) means (12×N)
    My = np.vstack([Yv[m_idx==k,:].mean(axis=0)   for k in range(12)])
    Mh = np.vstack([HDDv[m_idx==k,:].mean(axis=0) for k in range(12)])
    Mc = np.vstack([CDDv[m_idx==k,:].mean(axis=0) for k in range(12)])

    # centered pooled seasonal intercepts
    Hy0, Dy0 = pooled_tables_centered(Yv, hod, dow)

    # within (i,month) for Y and X; subtract pooled seasonal from Y only
    Yw   = (Yv   - My[m_idx, :]) - Hy0[hod, :] - Dy0[dow, :]
    HDDw = (HDDv - Mh[m_idx, :])
    CDDw = (CDDv - Mc[m_idx, :])

    # 2×2 normal equations
    Shh = np.einsum('ij,ij->', HDDw, HDDw); Scc = np.einsum('ij,ij->', CDDw, CDDw)
    Shc = np.einsum('ij,ij->', HDDw, CDDw)
    Shy = np.einsum('ij,ij->', HDDw, Yw);   Scy = np.einsum('ij,ij->', CDDw, Yw)
    det = Shh*Scc - Shc*Shc
    beta_B = np.array([(Shy*Scc - Scy*Shc)/det, (-Shy*Shc + Scy*Shh)/det], dtype=np.float64)

    # α_{i,m} on original scale
    alpha_im_B = (My - (beta_B[0]*Mh + beta_B[1]*Mc)).astype(np.float64)

    seasonal = {"Hy0": Hy0, "Dy0": Dy0}
    return alpha_im_B, beta_B, seasonal

alpha_im_B, beta_B, seasonal = fit_model_B_with_pooled_seasonal(Yv, HDDv, CDDv, m_idx, hod, dow)

In [12]:

def build_ixhour_centered(Yv, m_idx, hod):
    """
    Per-consumer hour-of-day intercepts (24×N), centered per consumer.
    We remove (i×month) first so 'hour' captures intra-month daily shape.
    """
    # (i×month) means
    My = np.vstack([Yv[m_idx==k, :].mean(axis=0) for k in range(12)])  # (12,N)
    Y_minus_month = Yv - My[m_idx, :]
    Hy_i = np.vstack([Y_minus_month[hod==h, :].mean(axis=0) for h in range(24)])  # (24,N)
    Hy_i0 = Hy_i - Hy_i.mean(axis=0, keepdims=True)  # center per consumer
    return Hy_i0, My

def build_ixdow_centered(residual_like, dow):
    """
    Per-consumer day-of-week intercepts (7×N), centered per consumer.
    Pass a 'residual-like' array from which DOW should explain a mean pattern
    (e.g., Y minus i×month, i×hour, and β·X).
    """
    Dy_i = np.vstack([residual_like[dow==d, :].mean(axis=0) for d in range(7)])  # (7,N)
    Dy_i0 = Dy_i - Dy_i.mean(axis=0, keepdims=True)
    return Dy_i0


In [13]:
def fit_model_B_ixhour(Yv, HDDv, CDDv, m_idx, hod, dow, Dy0=None):
    """
    Model B: (i×month) + (i×hour) + [pooled DOW optional], iid errors.
    Returns:
      alpha_im_B : (12,N)
      alpha_ih_B : (24,N)
      beta_B     : (2,)
      seasonal   : dict with {"Dy": Dy_used} for prediction
    """
    # (i×hour) centered + (i×month) tables
    alpha_ih_B, My = build_ixhour_centered(Yv, m_idx, hod)

    # month means for regressors
    Mh = np.vstack([HDDv[m_idx==k, :].mean(axis=0) for k in range(12)])
    Mc = np.vstack([CDDv[m_idx==k, :].mean(axis=0) for k in range(12)])

    # pooled DOW (centered) if not provided
    if Dy0 is None:
        Dy = np.vstack([Yv[dow==d, :].mean(axis=0) for d in range(7)])
        Dy0 = Dy - Dy.mean(axis=0, keepdims=True)

    # within transform for Y and X:
    #  - remove (i×month) from Y and X
    #  - remove i×hour from Y only
    #  - remove pooled DOW (centered) from Y only (optional)
    Yw   = (Yv   - My[m_idx, :] - alpha_ih_B[hod, :] - Dy0[dow, :])
    HDDw = (HDDv - Mh[m_idx, :])
    CDDw = (CDDv - Mc[m_idx, :])

    # 2×2 OLS (pooled)
    Shh = np.einsum('ij,ij->', HDDw, HDDw); Scc = np.einsum('ij,ij->', CDDw, CDDw)
    Shc = np.einsum('ij,ij->', HDDw, CDDw)
    Shy = np.einsum('ij,ij->', HDDw, Yw);   Scy = np.einsum('ij,ij->', CDDw, Yw)
    det = Shh*Scc - Shc*Shc
    beta_B = np.array([(Shy*Scc - Scy*Shc)/det,
                       (-Shy*Shc + Scy*Shh)/det], dtype=np.float64)

    # reconstruct (i×month) intercepts on original scale
    alpha_im_B = (My - (beta_B[0]*Mh + beta_B[1]*Mc)).astype(np.float64)

    # re-estimate i×hour intercepts from residual means for clean partition
    res_hour = Yv - (alpha_im_B[m_idx, :] + beta_B[0]*HDDv + beta_B[1]*CDDv + Dy0[dow, :])
    alpha_ih_B = np.vstack([res_hour[hod==h, :].mean(axis=0) for h in range(24)])
    alpha_ih_B = alpha_ih_B - alpha_ih_B.mean(axis=0, keepdims=True)

    seasonal = {"Dy": Dy0}
    return alpha_im_B, alpha_ih_B, beta_B, seasonal
    
alpha_im_B, alpha_ih_B, beta_B, seasonal = fit_model_B_ixhour(Yv, HDDv, CDDv, m_idx, hod, dow, Dy0=None)

In [14]:
print_coeffs_and_forecast_metrics(
    Yv, HDDv, CDDv, m_idx, beta_B,
    alpha_im=alpha_im_B, alpha_ih=alpha_ih_B, Dy=seasonal["Dy"], hod=hod, dow=dow,
    label="Model B (i×month + i×hour + pooled DOW, iid)"
)


Model B (i×month + i×hour + pooled DOW, iid)
--------------------------------------------
β_HDD = 0.0262
β_CDD = 0.0088
Static (mean) fit:
  R²   = 0.5327
  RMSE = 0.6158


### Model C — Month x Individual FE + correlated errors

cons ~ temp_low + temp_high

In [45]:
def fit_ar1_sar24_from_B(Yv, HDDv, CDDv, m_idx, alpha_im_B, beta_B):
    """
    Fit pooled AR(1)+SAR(24) residual process on Model B residuals.
    Returns (phi1, phi24, sigma_e).
    Model: e_t = phi1 * e_{t-1} + phi24 * e_{t-24} + eps_t
    """
    base_B = alpha_im_B[m_idx, :] + beta_B[0]*HDDv + beta_B[1]*CDDv
    r = Yv - base_B
    y   = r[24:, :]
    x1  = r[23:-1, :]
    x24 = r[:-24, :]

    S11 = np.einsum('ij,ij->', x1, x1)
    S22 = np.einsum('ij,ij->', x24, x24)
    S12 = np.einsum('ij,ij->', x1, x24)
    Sy1 = np.einsum('ij,ij->', x1, y)
    Sy2 = np.einsum('ij,ij->', x24, y)
    det = S11*S22 - S12*S12 + 1e-12

    phi1  = float(np.clip(( Sy1*S22 - Sy2*S12)/det,  -0.98, 0.98))
    phi24 = float(np.clip((-Sy1*S12 + Sy2*S11)/det, -0.98, 0.98))

    yhat = phi1*x1 + phi24*x24
    sigma_e = float(np.sqrt(np.mean((y - yhat)**2)))
    return phi1, phi24, sigma_e


phi1, phi24, sigma_e = fit_ar1_sar24_from_B(Yv, HDDv, CDDv, m_idx, alpha_im_B, beta_B)

In [17]:
def fit_model_C_iHour_mean(Yv, HDDv, CDDv, m_idx, hod, dow=None, use_pooled_dow=False):
    """
    Mean: (i×month) + (i×hour) [+ optional pooled DOW]; slopes pooled (β_HDD, β_CDD).
    Returns:
      alpha_im_C : (12,N)
      alpha_ih_C : (24,N)  (centered per consumer)
      beta_C     : (2,)
    """
    # per-(i,month) means (12×N) on Y and X
    months = np.arange(12)
    My = np.vstack([Yv[m_idx==k,:].mean(axis=0)   for k in months])
    Mh = np.vstack([HDDv[m_idx==k,:].mean(axis=0) for k in months])
    Mc = np.vstack([CDDv[m_idx==k,:].mean(axis=0) for k in months])

    # provisional i×hour from Y after removing (i×month); center per consumer
    Y_minus_month = Yv - My[m_idx, :]
    Hy_i = np.vstack([Y_minus_month[hod==h,:].mean(axis=0) for h in range(24)])
    alpha_ih_C = Hy_i - Hy_i.mean(axis=0, keepdims=True)

    # optional pooled DOW (centered)
    Dy0 = None
    if use_pooled_dow and (dow is not None):
        Dy = np.vstack([Yv[dow==d,:].mean(axis=0) for d in range(7)])
        Dy0 = Dy - Dy.mean(axis=0, keepdims=True)

    # within for β: remove (i×month) & i×hour from Y; remove (i×month) from X
    Yw   = Yv   - My[m_idx, :] - alpha_ih_C[hod, :]
    HDDw = HDDv - Mh[m_idx, :]
    CDDw = CDDv - Mc[m_idx, :]
    if Dy0 is not None:
        Yw -= Dy0[dow, :]

    # 2×2 solve
    Shh = np.einsum('ij,ij->', HDDw, HDDw); Scc = np.einsum('ij,ij->', CDDw, CDDw)
    Shc = np.einsum('ij,ij->', HDDw, CDDw)
    Shy = np.einsum('ij,ij->', HDDw, Yw);   Scy = np.einsum('ij,ij->', CDDw, Yw)
    det = Shh*Scc - Shc*Shc
    beta_C = np.array([(Shy*Scc - Scy*Shc)/det, (-Shy*Shc + Scy*Shh)/det], dtype=np.float64)

    # α_{i,m} on original scale (keep α_{i,h} as intercepts)
    alpha_im_C = (My - (beta_C[0]*Mh + beta_C[1]*Mc)).astype(np.float64)

    # refine α_{i,h} from residuals (on original scale), center again
    res = Yv - (alpha_im_C[m_idx,:] + beta_C[0]*HDDv + beta_C[1]*CDDv)
    if Dy0 is not None: res -= Dy0[dow,:]
    alpha_ih_C = np.vstack([res[hod==h,:].mean(axis=0) for h in range(24)])
    alpha_ih_C -= alpha_ih_C.mean(axis=0, keepdims=True)

    return alpha_im_C, alpha_ih_C, beta_C, Dy0


In [18]:
def fit_pooled_ar1_from_C_iHour(Yv, HDDv, CDDv, m_idx, hod, alpha_im_C, alpha_ih_C, beta_C, Dy0=None, dow=None):
    """
    Estimate pooled AR(1): e_t = ρ e_{t-1} + ε_t on residuals of the i×hour mean.
    Returns (rho, sigma_e).
    """
    mu = alpha_im_C[m_idx,:] + alpha_ih_C[hod,:] + beta_C[0]*HDDv + beta_C[1]*CDDv
    if (Dy0 is not None) and (dow is not None):
        mu = mu + Dy0[dow,:]
    res = Yv - mu
    num = np.einsum('ij,ij->', res[1:,:], res[:-1,:])
    den = np.einsum('ij,ij->', res[:-1,:], res[:-1,:]) + 1e-12
    rho = float(np.clip(num/den, -0.98, 0.98))
    innov = res[1:,:] - rho*res[:-1,:]
    sigma_e = float(np.sqrt(np.mean(innov**2)))
    return rho, sigma_e


In [19]:
def gls_refit_C_ar1_iHour(Yv, HDDv, CDDv, m_idx, hod, rho):
    """
    GLS refit of β under pooled AR(1) errors with i×hour in the mean.
    Transform:
      y*_t = y_t - ρ y_{t-1}; X*_t = X_t - ρ X_{t-1}
    Then within on (i×month) for X, and remove (i×month + i×hour) from y*.
    Rebuild α_{i,m} and α_{i,h} on the original (untransformed) scale.
    """
    # AR(1) transform (drop first row)
    Yco   = Yv[1:,:]   - rho*Yv[:-1,:]
    HDDco = HDDv[1:,:] - rho*HDDv[:-1,:]
    CDDco = CDDv[1:,:] - rho*CDDv[:-1,:]
    m_idx_co = m_idx[1:]
    hod_co   = hod[1:]
    months = np.arange(12)

    # month means on transformed arrays
    Myco = np.vstack([Yco[m_idx_co==k,:].mean(axis=0)   for k in months])
    Mhco = np.vstack([HDDco[m_idx_co==k,:].mean(axis=0) for k in months])
    Mcco = np.vstack([CDDco[m_idx_co==k,:].mean(axis=0) for k in months])

    # i×hour means on transformed Y only (center per consumer)
    Hyco = np.vstack([Yco[hod_co==h,:].mean(axis=0) for h in range(24)])
    Hyco -= Hyco.mean(axis=0, keepdims=True)

    # within: remove (i×month) from X*, and (i×month + i×hour) from y*
    Yw   = Yco   - Myco[m_idx_co,:] - Hyco[hod_co,:]
    HDDw = HDDco - Mhco[m_idx_co,:]
    CDDw = CDDco - Mcco[m_idx_co,:]

    # 2×2 GLS on transformed-within arrays
    Shh = np.einsum('ij,ij->', HDDw, HDDw); Scc = np.einsum('ij,ij->', CDDw, CDDw)
    Shc = np.einsum('ij,ij->', HDDw, CDDw)
    Shy = np.einsum('ij,ij->', HDDw, Yw);   Scy = np.einsum('ij,ij->', CDDw, Yw)
    det = Shh*Scc - Shc*Shc
    beta_C = np.array([(Shy*Scc - Scy*Shc)/det, (-Shy*Shc + Scy*Shh)/det], dtype=np.float64)

    # Rebuild α on original scale (untransformed monthly/hour means)
    # First, get untransformed month means (already have Mh, Mc on original if stored;
    # recompute here to be self-contained)
    # (12×N) on original
    # -- month means for Y,X on original scale
    My = np.vstack([Yv[m_idx==k,:].mean(axis=0)   for k in months])
    Mh = np.vstack([HDDv[m_idx==k,:].mean(axis=0) for k in months])
    Mc = np.vstack([CDDv[m_idx==k,:].mean(axis=0) for k in months])

    alpha_im_C = (My - (beta_C[0]*Mh + beta_C[1]*Mc)).astype(np.float64)

    # α_{i,h} from original residual means; center per consumer
    res_for_hour = Yv - (alpha_im_C[m_idx,:] + beta_C[0]*HDDv + beta_C[1]*CDDv)
    alpha_ih_C = np.vstack([res_for_hour[hod==h,:].mean(axis=0) for h in range(24)])
    alpha_ih_C -= alpha_ih_C.mean(axis=0, keepdims=True)

    return alpha_im_C, alpha_ih_C, beta_C


In [20]:
# Indices
Tn, N = Yv.shape
idx = np.arange(Tn)
hod = idx % 24
dow = (idx // 24) % 7

# 1) Fit mean with i×hour (optionally pooled DOW in mean; set use_pooled_dow=True if you want it)
alpha_im_C, alpha_ih_C, beta_C, Dy0 = fit_model_C_iHour_mean(Yv, HDDv, CDDv, m_idx, hod, dow, use_pooled_dow=False)

# 2) Fit pooled AR(1) on residuals of that mean
rho, sigma_e = fit_pooled_ar1_from_C_iHour(Yv, HDDv, CDDv, m_idx, hod, alpha_im_C, alpha_ih_C, beta_C, Dy0=None, dow=dow)

# 3) Optional GLS refit under AR(1)
alpha_im_C, alpha_ih_C, beta_C = gls_refit_C_ar1_iHour(Yv, HDDv, CDDv, m_idx, hod, rho)

# 4) Evaluate (your universal printer already handles this)
print_coeffs_and_forecast_metrics(
    Yv, HDDv, CDDv, m_idx, beta_C,
    alpha_im=alpha_im_C, alpha_ih=alpha_ih_C,
    hod=hod, dow=dow,   # dow only used if you also add pooled/i×DOW; otherwise harmless
    rho=rho,
    label="Model C (i×month + i×hour mean; pooled AR(1) errors)"
)



Model C (i×month + i×hour mean; pooled AR(1) errors)
----------------------------------------------------
β_HDD = 0.0257
β_CDD = 0.0086
Static (mean) fit:
  R²   = 0.5311
  RMSE = 0.6169

Dynamic one-step forecast (AR1):
  ρ    = 0.3186
  R²   = 0.5787
  RMSE = 0.5847


In [46]:
def gls_refit_C_ar1_sar24(Yv, HDDv, CDDv, m_idx, phi1, phi24, MyMhMc):
    """
    One-shot GLS refit of beta under AR(1)+SAR(24) residuals.
    Transform:
      y*_t   = y_t   - phi1*y_{t-1}   - phi24*y_{t-24}
      X*_t   = X_t   - phi1*X_{t-1}   - phi24*X_{t-24}
    Drop first 24 rows, then within-(i,m) OLS on transformed arrays.
    Returns:
      alpha_im_C : (12, N) month×individual intercepts (on original scale)
      beta_C     : (2,)    [β_HDD, β_CDD]
    """
    My, Mh, Mc = MyMhMc

    y_co   = Yv[24:, :]   - phi1*Yv[23:-1, :]   - phi24*Yv[:-24, :]
    hdd_co = HDDv[24:, :] - phi1*HDDv[23:-1, :] - phi24*HDDv[:-24, :]
    cdd_co = CDDv[24:, :] - phi1*CDDv[23:-1, :] - phi24*CDDv[:-24, :]
    m_idx_co = m_idx[24:]

    months = np.arange(12)
    Myco = np.vstack([y_co[m_idx_co==k,   :].mean(axis=0)   for k in months])
    Mhco = np.vstack([hdd_co[m_idx_co==k, :].mean(axis=0)   for k in months])
    Mcco = np.vstack([cdd_co[m_idx_co==k, :].mean(axis=0)   for k in months])

    ycw   = y_co   - Myco[m_idx_co, :]
    hdcow = hdd_co - Mhco[m_idx_co, :]
    cdcow = cdd_co - Mcco[m_idx_co, :]

    Shh = np.einsum('ij,ij->', hdcow, hdcow, optimize=True)
    Scc = np.einsum('ij,ij->', cdcow, cdcow, optimize=True)
    Shc = np.einsum('ij,ij->', hdcow, cdcow, optimize=True)
    Shy = np.einsum('ij,ij->', hdcow, ycw,   optimize=True)
    Scy = np.einsum('ij,ij->', cdcow, ycw,   optimize=True)
    det = Shh*Scc - Shc*Shc
    beta_C = np.array([( Shy*Scc - Scy*Shc)/det,
                       (-Shy*Shc + Scy*Shh)/det], dtype=np.float64)

    # Recompute α_{i,m} on *untransformed* monthly means
    alpha_im_C = (My - (beta_C[0]*Mh + beta_C[1]*Mc)).astype(np.float64)
    return alpha_im_C, beta_C

alpha_im_C, beta_C = gls_refit_C_ar1_sar24(Yv, HDDv, CDDv, m_idx, phi1, phi24, MyMhMc)

In [48]:
def print_coeffs_and_forecast_metrics_ar1_sar24(
    Yv, HDDv, CDDv, m_idx, beta, alpha_im, phi1=None, phi24=None,
    label="Model C (AR(1)+SAR(24))"
):
    """
    Prints β, static R²/RMSE (mean fit), and dynamic one-step R²/RMSE
    using realized residuals with AR(1)+SAR(24) dynamics if φ's provided.
    """
    mu = alpha_im[m_idx, :] + beta[0]*HDDv + beta[1]*CDDv

    # Static (mean) fit
    resid_s = Yv - mu
    sse_s = float(np.sum(resid_s**2))
    sst   = float(np.sum((Yv - Yv.mean())**2))
    r2_s  = 1.0 - sse_s/sst
    rmse_s = float(np.sqrt(sse_s / Yv.size))

    print(f"\n{label}")
    print("-"*len(label))
    print(f"β_HDD = {beta[0]:.4f}")
    print(f"β_CDD = {beta[1]:.4f}")
    print("Static (mean) fit:")
    print(f"  R²   = {r2_s:.4f}")
    print(f"  RMSE = {rmse_s:.4f}")

    # Dynamic one-step (evaluation)
    if (phi1 is not None) and (phi24 is not None):
        yhat_dyn = predict_dynamic_one_step_ar1_sar24(Yv, mu, phi1, phi24)
        resid_d = Yv - yhat_dyn
        sse_d = float(np.sum(resid_d**2))
        r2_d  = 1.0 - sse_d/sst
        rmse_d = float(np.sqrt(sse_d / Yv.size))
        print("\nDynamic one-step forecast (AR(1)+SAR(24)):")
        print(f"  φ1   = {phi1:.4f}")
        print(f"  φ24  = {phi24:.4f}")
        print(f"  R²   = {r2_d:.4f}")
        print(f"  RMSE = {rmse_d:.4f}")

In [50]:
print_coeffs_and_forecast_metrics_ar1_sar24(
    Yv, HDDv, CDDv, m_idx, beta_C, alpha_im_C, phi1, phi24,
    label="Model C (AR(1)+SAR(24))"
)


Model C (AR(1)+SAR(24))
-----------------------
β_HDD = 0.0117
β_CDD = -0.0005
Static (mean) fit:
  R²   = 0.3294
  RMSE = 0.7377

Dynamic one-step forecast (AR(1)+SAR(24)):
  φ1   = 0.2494
  φ24  = 0.5644
  R²   = 0.6421
  RMSE = 0.5390


In [51]:
def fit_pooled_ar1_from_B(Yv, HDDv, CDDv, m_idx, alpha_im_B, beta_B):
    base_B = alpha_im_B[m_idx, :] + beta_B[0]*HDDv + beta_B[1]*CDDv
    res = Yv - base_B
    # pooled ρ
    num = np.einsum('ij,ij->', res[1:,:], res[:-1,:])
    den = np.einsum('ij,ij->', res[:-1,:], res[:-1,:]) + 1e-12
    rho = float(np.clip(num/den, -0.98, 0.98))
    # innovation std
    innov = res[1:,:] - rho*res[:-1,:]
    sigma_e = float(np.sqrt(np.mean(innov**2)))
    return rho, sigma_e
rho, sigma_e = fit_pooled_ar1_from_B(Yv, HDDv, CDDv, m_idx, alpha_im_B, beta_B)

In [52]:
def gls_refit_C(Yv, HDDv, CDDv, m_idx, rho, MyMhMc):
    My, Mh, Mc = MyMhMc
    # Cochrane–Orcutt transform
    Yco   = Yv[1:,:]   - rho*Yv[:-1,:]
    HDDco = HDDv[1:,:] - rho*HDDv[:-1,:]
    CDDco = CDDv[1:,:] - rho*CDDv[:-1,:]
    m_idx_co = m_idx[1:]

    # monthly means of transformed vars
    months = np.arange(12)
    Myco = np.vstack([Yco[m_idx_co==k,:].mean(axis=0)   for k in months])
    Mhco = np.vstack([HDDco[m_idx_co==k,:].mean(axis=0) for k in months])
    Mcco = np.vstack([CDDco[m_idx_co==k,:].mean(axis=0) for k in months])

    # within on transformed arrays
    Yco_w   = Yco   - Myco[m_idx_co, :]
    HDDco_w = HDDco - Mhco[m_idx_co, :]
    CDDco_w = CDDco - Mcco[m_idx_co, :]

    # 2×2 GLS normal equations
    Shh = np.einsum('ij,ij->', HDDco_w, HDDco_w, optimize=True)
    Scc = np.einsum('ij,ij->', CDDco_w, CDDco_w, optimize=True)
    Shc = np.einsum('ij,ij->', HDDco_w, CDDco_w, optimize=True)
    Shy = np.einsum('ij,ij->', HDDco_w, Yco_w,   optimize=True)
    Scy = np.einsum('ij,ij->', CDDco_w, Yco_w,   optimize=True)
    det = Shh*Scc - Shc*Shc
    beta_C = np.array([( Shy*Scc - Scy*Shc)/det,
                       (-Shy*Shc + Scy*Shh)/det], dtype=np.float64)

    # α_{i,m} recomputed on *untransformed* monthly means
    alpha_im_C = (My - (beta_C[0]*Mh + beta_C[1]*Mc)).astype(np.float64)
    return alpha_im_C, beta_C
alpha_im_C, beta_C = gls_refit_C(Yv, HDDv, CDDv, m_idx, rho, MyMhMc)

In [53]:
def predict_mean(HDDv, CDDv, m_idx, beta, alpha_im):
    return alpha_im[m_idx, :] + beta[0]*HDDv + beta[1]*CDDv

def predict_dynamic_one_step(Yv, mean_static, rho):
    # ŷ_t = μ_t + ρ * (y_{t-1} - μ_{t-1})
    yhat = mean_static.copy()
    yhat[1:,:] += rho * (Yv[:-1,:] - mean_static[:-1,:])
    return yhat


In [55]:
def print_coeffs_and_forecast_metrics_pooledAR1(
    Yv, HDDv, CDDv, m_idx, beta, alpha_im, rho=None, label="Model"
):
    """
    Print coefficients, static fit metrics, and dynamic (one-step) forecast metrics.
    Now includes R² for both static and dynamic fits.
    """
    # --- Static mean prediction ---
    mean_static = predict_mean(HDDv, CDDv, m_idx, beta, alpha_im)
    resid_static = Yv - mean_static

    sse_s = float(np.sum(resid_static**2))
    sst   = float(np.sum((Yv - Yv.mean())**2))
    r2_s  = 1.0 - sse_s/sst
    rmse_s = float(np.sqrt(sse_s / Yv.size))

    # --- Dynamic one-step-ahead prediction ---
    if rho is None:
        yhat_dyn = mean_static
    else:
        yhat_dyn = predict_dynamic_one_step(Yv, mean_static, rho)
    resid_dyn = Yv - yhat_dyn

    sse_d = float(np.sum(resid_dyn**2))
    r2_d  = 1.0 - sse_d/sst          # same total variance denominator
    rmse_d = float(np.sqrt(sse_d / Yv.size))

    # --- Print summary ---
    print(f"\n{label}")
    print("-"*len(label))
    print(f"β_HDD = {beta[0]:.4f}")
    print(f"β_CDD = {beta[1]:.4f}")

    print("\nStatic (mean) fit:")
    print(f"  R²   = {r2_s:.4f}")
    print(f"  RMSE = {rmse_s:.4f}")

    if rho is not None:
        print("\nDynamic one-step forecast (pooled AR1):")
        print(f"  ρ    = {rho:.4f}")
        print(f"  R²   = {r2_d:.4f}")
        print(f"  RMSE = {rmse_d:.4f}")


print_coeffs_and_forecast_metrics_pooledAR1(
    Yv, HDDv, CDDv, m_idx, beta_C, alpha_im_C, rho=rho, label="Model C"
)


Model C
-------
β_HDD = 0.0312
β_CDD = -0.0042

Static (mean) fit:
  R²   = 0.3346
  RMSE = 0.7349

Dynamic one-step forecast (pooled AR1):
  ρ    = 0.4117
  R²   = 0.4474
  RMSE = 0.6697


### Diagnostics 

In [56]:
mu = alpha_im_C[m_idx,:] + beta_C[0]*HDDv + beta_C[1]*CDDv
num = np.sum((Yv - mu)**2, axis=0)
den = np.sum((Yv - Yv.mean(axis=0, keepdims=True))**2, axis=0)
r2_i = 1 - num/den
np.nanquantile(r2_i, [0.1,0.5,0.9])


array([-0.01868965,  0.11543623,  0.49912783])